In [1]:
import numpy as np
import imageio
from skimage import transform
import matplotlib.pyplot as plt
from math import sqrt
import glob
import subprocess

%matplotlib inline

In [2]:
import librosa

In [44]:
p = subprocess.Popen("ffmpeg -i data/videos/animals/2.mp4 -f wav -vn data/tmp/animals_0.wav", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

Checking if there is an audio channel. If not, the output audio stream will be empty.

In [48]:
p = subprocess.Popen("ffprobe -i data/videos/animals/0.mp4 -show_streams -select_streams a -loglevel error"\
                     , stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [61]:
"No audio channel" if output.decode()=='' else "Audio channels present:"+output.decode()

No audio channel


In [62]:
!ffprobe -i data/videos/animals/0.mp4 -show_streams -select_streams a -loglevel error

In [66]:
p = subprocess.Popen("ffprobe -i /home/hemant/Videos/Titans\ -\ Season\ 1.mp4 -show_streams -select_streams a -loglevel error", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [71]:
"No audio channel" if output.decode()=='' else "Audio channels present:"+output.decode()

'Audio channels present:[STREAM]\nindex=1\ncodec_name=aac\ncodec_long_name=AAC (Advanced Audio Coding)\nprofile=LC\ncodec_type=audio\ncodec_time_base=1/44100\ncodec_tag_string=mp4a\ncodec_tag=0x6134706d\nsample_fmt=fltp\nsample_rate=44100\nchannels=2\nchannel_layout=stereo\nbits_per_sample=0\nid=N/A\nr_frame_rate=0/0\navg_frame_rate=0/0\ntime_base=1/44100\nstart_pts=0\nstart_time=0.000000\nduration_ts=114762752\nduration=2602.329977\nbit_rate=125588\nmax_bit_rate=N/A\nbits_per_raw_sample=N/A\nnb_frames=112073\nnb_read_frames=N/A\nnb_read_packets=N/A\nDISPOSITION:default=1\nDISPOSITION:dub=0\nDISPOSITION:original=0\nDISPOSITION:comment=0\nDISPOSITION:lyrics=0\nDISPOSITION:karaoke=0\nDISPOSITION:forced=0\nDISPOSITION:hearing_impaired=0\nDISPOSITION:visual_impaired=0\nDISPOSITION:clean_effects=0\nDISPOSITION:attached_pic=0\nDISPOSITION:timed_thumbnails=0\nTAG:creation_time=2018-12-03T23:09:27.000000Z\nTAG:language=eng\nTAG:handler_name=ISO Media file produced by Google Inc. Created on: 12

In [72]:
p = subprocess.Popen("ffmpeg -i /home/hemant/Videos/Titans\ -\ Season\ 1.mp4 -f wav -vn data/tmp/animals_0.wav", \
                     stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

In [77]:
# The file is huge. Taking a small chunk of it.
!ffmpeg -t 30 -i data/tmp/animals_0.wav data/tmp/mini_0.wav

ffmpeg version 3.4 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 7.2.0 (crosstool-NG fa8859cb)
  configuration: --prefix=/home/hemant/anaconda3 --disable-doc --enable-shared --extra-cflags='-fPIC -I/home/hemant/anaconda3/include' --extra-cxxflags='=-fPIC' --extra-libs='-L/home/hemant/anaconda3/lib -lz' --enable-pic --disable-static --disable-gpl --disable-nonfree --disable-openssl --enable-libvpx --cc=/opt/conda/conda-bld/ffmpeg_1530807717919/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --cxx=/opt/conda/conda-bld/ffmpeg_1530807717919/_build_env/bin/x86_64-conda_cos6-linux-gnu-c++ --enable-libopus
  libavutil      55. 78.100 / 55. 78.100
  libavcodec     57.107.100 / 57.107.100
  libavformat    57. 83.100 / 57. 83.100
  libavdevice    57. 10.100 / 57. 10.100
  libavfilter     6.107.100 /  6.107.100
  libswscale      4.  8.100 /  4.  8.100
  libswresample   2.  9.100 /  2.  9.100
Guessed Channel Layout for Input Stream #0.0 : stereo
Input #0, wav, from 'data/tmp/animals

In [78]:
from scipy.io import wavfile
from scipy import signal

sample_rate=16000
window_size=20
step_size=10
eps=1e-10
rate, data = wavfile.read('data/tmp/mini_0.wav')
if data.ndim > 1 : # ignore  channels 2+
    data = data[:, 0]
nperseg = int(round(window_size * sample_rate / 1e3))
noverlap = int(round(step_size * sample_rate / 1e3))
freqs, times, spec = signal.spectrogram(data,fs=sample_rate,window='hann',nperseg=nperseg,noverlap=noverlap)
log_specgram = np.log(spec.T.astype(np.float32) + eps)

In [80]:
log_specgram.shape

(8267, 161)